# CNN Tutorial

Written by [Robert Morgan](https://rmorgan10.github.io/), and based on content from [Brian Nord](http://briandnord.com/), [Joshua Yao-Yu Lin](https://joshualincosmo.wordpress.com/), and [Sebastian Raschka](https://sebastianraschka.com/).

Let's make a CNN!

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
from pycm import ConfusionMatrix
from scipy import misc
from scipy import ndimage

Enter `pytorch`. PyTorch will be the main workhorse behind our deep learning adventure. Some people use Keras and Tensorflow, but that's not what we'll cover in this tutorial.

I chose PyTorch because I believe it provides a lot of functionality that Keras does not. Keras was designed to act as an outer shell for the main deep learning engine tensorflow. While this made the utilities of tensorflow more accessible, it makes it difficult to intorduce sophisticated neural network architectures because you are not interacting directly with tensorflow. PyTorch allows you to have full control over your neural network while (in my opinion) being much easier to interact with than tensorflow.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
import torchvision

Let's load in some toy data. The MNIST dataset is a collection of 60,000 labelled, hand-written digits in 28x28 arrays.

In [ ]:
if not os.path.exists('CNN_Tutorial_data/'):
    os.mkdir('CNN_Tutorial_data/')
    
if not os.path.exists('CNN_Tutorial_data/mnist/'):
    os.mkdir('CNN_Tutorial_data/mnist')
    DOWNLOAD_MNIST = True
else:
    DOWNLOAD_MNIST = False
    
data = torchvision.datasets.MNIST(root='CNN_Tutorial_data/mnist', 
                                  transform=torchvision.transforms.ToTensor(),
                                  download=DOWNLOAD_MNIST)


**What did that do?**

In the top half of the cell, we make sure directories exist for storing the MNIST dataset on your computer and determine if the dataset needs to be downloaded or if it already exists.

In the bottom half of the cell, we load the MNIST dataset into the variable `data`. The data is stored in the `root` directory, so in our case this is `CNN_Tutorial_data/mnist/`. The `transform` argument converts the input data format to a `Tensor` object, which is PyTorch's way of processing things.

### Preprocessing the MNIST dataset

First, let's take a look at the data so we know what we are doing.

The information stored in `data` has the following attributes:

In [ ]:
print([x for x in dir(data)])

So it looks like our data and their labels are likely in `data.train_data` and `data.train_labels`.

To check that guess, let's look at the shapes of these things, and maybe plot a couple.

In [ ]:
print(data.train_data.shape)
print(data.train_labels.shape)

In [ ]:
fig, axs = plt.subplots(1,5, figsize=(18, 5))

for i in range(len(axs)):
    axs[i].imshow(data.train_data[i], cmap='gray')
    axs[i].set_title('MNIST Label: {0}'.format(data.train_labels[i]), fontsize=16)
plt.show()

With images, it's also a good idea to look at the actual pixel values:

In [ ]:
print(data.train_data[0])

Kinda neat how you can almost detect the pattern in the raw pixel values. Anyways, the individual pixels are valued between 0 and 255, and appear to be `int`s. The neural net we will build will multiply weights by the individual pixel values. Therefore, to avoid the products getting too large or the weights getting very small during training, it is standard practice to noramlize the pixel counts to be `float`s between 0.0 and 1.0.

Luckily, PyTorch can do this for us. We already specified that we wanted `transform=torchvision.transforms.ToTensor()` when importing the data, and this transform tells PyTorch to scale the pixel values to be between 0.0 and 1.0 when training.

**Note:** If your normalization scheme does not match between your training and testing sets, your network will perform no better than random guessing. Checking that the noramlization has been done correctly is always worth your time.

### Desining a convolutional neural network

Our goal now will be to build a convolutional neural network that can take as input a 28x28 image array and correctly identify the handwritten digit based on patterns in the pixel counts.

In the PyTorch way of doing things, we will create a class for the classifier. The start of the object will look like this:

```python
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
```

But what does all that do? Let's break it down line by line.

```python 
class CNN(nn.Module):
``` 
This defines a new abstract object type `CNN`. Becasue of our definition, a `CNN` object can be thought of a similar to a `list` object or a `numpy.ndarray` object: it's just a framework that python will reconize and be able to store data in and work with. Adding `nn.Module` to the argument of the `CNN` object makes the object aware of the properties of `nn.Module`, which remember we imported from PyTorch.

```python
    def __init__(self):
```
The `__init__()` function is always the first line of every class. It is automatically executed when ever a `CNN` object is instantiated. It needs the self argument to access, the properties and attributes of the class you're defining, so always add that in.

```python
        super(CNN, self).__init__()
```
The `super()` function is a python built-in function. It takes whatever properties, methods, and attributes are defined in the arguments of `CNN` and adds them to `CNN` itself. In our case, the only argument of `CNN` is `nn.Module`, so all methods in `nn.Module` will now be a part of our `CNN` object.



---



If that crash course in object-oriented programming makes sense, then we can move on to using the `CNN` object to define the components and flow of our convolutional neural network. The next step is to define the internal components of the network. 

```python
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        #Network Components
        self.conv1 = nn.Conv2d(in_channels=1, 
                               out_channels=32, 
                               kernel_size=5, 
                               stride=1,
                               padding=2)
        
        self.conv2 = nn.Conv2d(in_channels=32, 
                               out_channels=64,
                               kernel_size=3, 
                               stride=1,
                               padding=2)
        
        self.dropout1 = nn.Dropout2d(0.25)
        
        self.dropout2 = nn.Dropout2d(0.5)
        
        self.fc1 = nn.Linear(in_features=14400, 
                             out_features=128)
        
        self.fc2 = nn.Linear(in_features=128, 
                             out_feature=10)

```
There's a lot in there. Let's unpack it.

**Convolutional Layers**

Convolutional layers are designed to create abstract representations of array-based data. They operate by applying kernels (sometimes called filters, which is super confusing when we try to use CNNs in astronomy), which are matrices of weights, to different parts of the image. Let's look at the convolutional layer illustration below to understand what some of the parameters mean.

<img src="./images/padding.png" alt="ljag" width="800"/>

[Image Source](https://medium.com/@pushpanjalipd/conventional-neural-network-deep-learning-a2ad1cf4487a)

`kernel_size`: In the above figure, `kernel_size`=2. This refers to the size of the blue matrix that is slid across the image, which is 2x2. Generally, you want the kernel size to reflect the size of the features in the image you are trying to detect. It is also useful to use a slightly smaller `kernel_size` in subsequent convolutional layers, since you will be extracting features from the products of the previous convolutional operator.

`stride`: Stride is the number of pixels the kernel is slid each time it is applied to the image. In our network, `stride`=1, so the kernels will move one pixel at a time.

`padding`: The padding in the above figure is set to 1, and is shown by the yellow boxes with 0 pixel counts. These are not part of the raw image, they are added onto the image by the CNN. Why? Well, let's think about the left border of the image. Without an added column of zeros, the right half of the kernel cannot access the pixel counts in the left border since the left half of the kernel would be outside the image. Adding a border of zeros allows your kernels to sample the entire image. The chosen witdth of the padding should take into account the `kernel_size`, `stride`, and image dimensions.

`in_channels`: This parameter refers to the number of filters feeding into the current convolutional layer. If it is the first layer in the network, the `in_channels` parameter should reflect the number of colors. For example, the MNIST dataset is grayscale, so we set `in_channels`=1. In astronomy, we have color information from optical bandpasses ($g$, $r$, $i$, $z$ for DECam), so the first layer may have `in_channels`=4. If this layer is not the first layer and is receiving input from a previous convolutional layer, set the number of `in_channels` to match the previous layer's number of `out_channels`.

`out_channels`: This parameter determines how many different convolutional kernels get applied. With low `kernel_size`, the different operators can become redundant. The chosen number of `out_channels` is usually tuned by hand.

**Dropout Layers**

Neural networks have a lot of parameters. A lot. If you look back at the `CNN` class, you can see that at one point, the network will have 14400 features! If the number of features begins to approach or exceeds the number of pixels in your images, you likely will want to consider a procedure known as dropout.

Dropout gives each internal connection in your network a probability (the `p` argument in the layer definition) of being removed from the network. By applying this probability randomly, the chances of the network being able to memorize images instead of lock on to real identifying features is reduced significantly. The machine learning way to phrase this concept is a means to "reduce overfitting."

**Fully-Connected Layers**

At some point, we have to go from abstracted representations of the image arrays to probabilities of each class. This is the goal of fully-connected layers.

The first step is to flatten whatever array your images are in. You can do this easily with the `torch.flatten()` method. As an example, this operation will take a 30x30 image array and convert to a length 900 vector. Your next goal is to reduce this length 900 vector to **a length equal to the number of classes in your problem**. This choice is made so that the network can straightforwardly compare its predictions to your data labels. For the MNSIT dataset we want to end up with 10 features, one for each class of hand-written digit.

You may also notice that in the network we do not perform the compression from 14400 features to 10 features in a single layer. As a general rule of thumb, a fully-connected layer performs well if its output size is roughly the square root of the input, but the number of fully-connected layers and their sizes is certainly something that can and should be tuned by hand to optimize your network.

One final important quesiton: how the heck did we get to 14400 features, and how do you know how many interal features your network will have? We'll answer this question in the next section where we look at the flow of data through the network.

<br>

---

<br>

The final portion of network design involves telling the network how to order the layers. This is done by adding a `forward()` method to the `CNN` class. This method should always be named `forward`, unless you choose to do some weird recurrent-like structure.

```python
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        #Network Components
        self.conv1 = nn.Conv2d(in_channels=1, 
                               out_channels=32, 
                               kernel_size=5, 
                               stride=1,
                               padding=2)
        
        self.conv2 = nn.Conv2d(in_channels=32, 
                               out_channels=64,
                               kernel_size=3, 
                               stride=1,
                               padding=2)
        
        self.dropout1 = nn.Dropout2d(0.25)
        
        self.dropout2 = nn.Dropout2d(0.5)
        
        self.fc1 = nn.Linear(in_features=14400, 
                             out_features=128)
        
        self.fc2 = nn.Linear(in_features=128, 
                             out_features=10)
        
    def forward(self, x):
        #Network Flow
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

```

Let's break down the `forward()` method. You can see that in each line, the infomation carried in the variable `x` is passed through one of the layers defined in the `CNN` object, and then the output of that layer is stored in `x` again and then sent to the next layer. As a technical note, the computations performed while training require a lot of memory on your CPU. To lighten the load a little bit, we overwrite the same variable so that a new memory address is not required for each layer output.

You will also notice there are some new items here: `max_pool2d`, `relu`, and `log_softmax`. `relu` and `softmax` are the names of activation functions. You can read about them [here](https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6). They are internal steps the network uses to determine whether a given abstract representation is of value, and they act in a way that turns on and off parts of the network based on the parts' importance in obtaining the right classificaiton.

`max_pool2d`: This type of layer is used to reduce the overall size of the images by looking at a section of an image, determining which part of it carries the most weight in making classifications, and then using only that part to represent the entire section. Here is an illustration to make that point clear:

<img src="./images/pooling.png" alt="ljag" width="600"/>

[Image Source](https://github.com/rasbt/stat479-deep-learning-ss19/blob/master/L13_intro-cnn/L13_intro-cnn-part1_slides.pdf)

In practice, `max_pool2d` layers tend to follow sets of convolutional layers, since they help to choose the most significant of the abstracted features produced during the convolution.

---

Let's briefly return to the issue of tracking the number of features through the network. And let's look at our network as a test case. We started with images of dimension 28x28. This is already 784 features, one for each pixel.

- Step 1: We applied a convolutional layer with `kernel_size`=5, `stride`=1, and `padding`=2. After a convolution with these properties, the new image widths are given by: $$ \textrm{New Width} = \frac{\textrm{Old Width} - \textrm{Kernel Width} + 2 \times \textrm{Padding}}{\textrm{Stride}} + 1,$$ so in our case the new image dimensions are 28x28. We are still at 784 features.

- Step 2: We apply a `relu` activation function, but this just applies an operation to each feature without combining any or producing any new ones, so we remain at 784 features.

- Step 3: We apply a second convolutional layer, this time with `kernel_size`=3, `stride`=1, and `padding`=2. Using the above formula, the new image widths will be 30x30, so we now have 900 features.

- Step 4: Time for `max_pool2d` with `kernel_size=2`, which compresses the images but does so by adding more weights to the network. Specifically, for each pixel in the image width, you apply a 2x2 kernel. So for an image of width 30, you are effectively lengthing it to $30 \times 2 \times 2$ = 120. Meaning as arrays we have 120x120 images, or 14400 features.

- Step 5: Dropout layers operate by removing individual connections at random, not by introducing new weights, so no new features are introduced here.

- Steps 6+: We now flatten the data arrays. So our 120x120 arrays are officially converted to length 14400 vectors and are ready for the fully-connected layers.

---

Now that we have looked at all the individual components of our neural network, lets put it all together in a cell that can be executed.

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        #Network Components
        self.conv1 = nn.Conv2d(in_channels=1, 
                               out_channels=32, 
                               kernel_size=5, 
                               stride=1,
                               padding=2)
        
        self.conv2 = nn.Conv2d(in_channels=32, 
                               out_channels=64,
                               kernel_size=3, 
                               stride=1,
                               padding=2)
        
        self.dropout1 = nn.Dropout2d(0.25)
        
        self.dropout2 = nn.Dropout2d(0.5)
        
        self.fc1 = nn.Linear(in_features=14400, 
                             out_features=128)
        
        self.fc2 = nn.Linear(in_features=128, 
                             out_features=10)
        
    def forward(self, x):
        #Network Flow
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

cnn = CNN()
print(cnn)

Cool. Now we have a neural network.

### Training the neural network

Next, we need to tell the neural network how to learn. We do this by specifying a loss function. While training, the neural network will make predictions about the input data and compare those predictions to the true labels of the data. If the neural network gets the answer wrong, then the interanl parameters of the neural network that led to the incorrect decision need to be updated. The amount that the interal parameters (weights) are updated after a wrong decision is what the loss function specifies. A standard loss function is catagorical cross entropy.

In [ ]:
loss_function = nn.CrossEntropyLoss()

The loss function defines a surface in a very high dimensional space--the space will have the same number of dimensions as the neural network has weights, so this can easily get to hundreds of thousands. Not surprisingly, figuring out which direction to move in this complicated space to update the weights can be computationally expensive. Thankfully, smart people have developed algorithms for moving through this space in efficient ways. To specify how you move along the loss surface in the space of the weights, you use something called an optimizer. The Adam optimizer is an industry standard.

Optimizers typically require some input parameters that tell them how fast to move along the loss surface. For the Adam optimizer, we have to specify something called the learning rate. Large learning rates mean big steps along the loss surface, so you will approach the optimal solution faster but risk overshooting it. Small learning rates mean tiny steps along the loss surface, so your overall training time will be slower, and as well you also risk getting stuck in a local minimum of the loss surface. In practice, this is a parameter you should tune by hand.

In [ ]:
learning_rate = 0.001
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

Now we're ready to train the neural network!

Two things we need to specify are how long we want to allow the neural network to train and how many images we want it to look at simultaneously. An epoch is how many times you want the network to find a minimum in the loss surface, adjust the weights to that position, and then start the trianing process over. Since time spent training the network is real time in your life, it would be ideal if this happened as quickly as possible, but too few training epochs and you may not reach the most optimal network configuration. A similar argument can be made for batch size, which is the number of images the network sees at once. If your batch size is too small, the weights may start to memorize the individual images in each batch rather than lock on to more global features, but too many images at once can make it difficult for the network to isolate the most informative features. Again, in practice, both of these parameters are tuned by hand.

In [ ]:
number_of_training_epochs = 3
batch_size = 50

As a good machine learning habbit, we should train our network on our training data, but evaluate it on unseen validation data. Our training data is ready to go, but we'll have to bring in and preprocess some new validation data.

In [ ]:
test_data = torchvision.datasets.MNIST(root='CNN_Tutorial_data/mnist/', train=False, download=True, transform=torchvision.transforms.ToTensor())
test_x = torch.unsqueeze(test_data.test_data, dim=1).type(torch.FloatTensor)[0:1000]
test_y = test_data.test_labels[0:1000]

We also need to prepare the training data to be loaded into the network in batches. In PyTorch, this process is done using a `DataLoader`.

In [ ]:
train_loader = Data.DataLoader(dataset=data, batch_size=batch_size, shuffle=True)

While training, we'll print out the network accuracy on small subsets of the training and testing data. If the performance is significantly different between these two datasets, or if the network performs at the random guessing level, scrutinize every detail of how you did the preprocessing.

Note: In the training below, the network will pause the training to make predictions every 10 steps for the purpose of nice(-ish) looking diagnostic plots later on. Making predictions every 10 steps slows the network down considerably. If you want to get through this step faster, change 

```python
if step % 10 == 0:
``` 

to 
```python
if step % 100 == 0:
```

LET'S GOOOO!

In [ ]:
# Append some lists as we go to track the accuracies and loss
indices, losses, train_acc, validation_acc = [], [], [], []
index_counter = 0


for epoch in range(number_of_training_epochs):
    
    for step, (batch_data, batch_labels) in enumerate(train_loader):
        
        #Clear out all existing gradients on the loss surface to reevaluate for this step
        optimizer.zero_grad()
        
        #Get the CNN's current prediction of the training data
        output = cnn(batch_data)
        
        #Calculate the loss by comparing the prediction to the truth
        loss = loss_function(output, batch_labels) 
        
        #Evaluate all gradients along the loss surface using back propagation
        loss.backward()
        
        #Based on the gradients, take the optimal step in the weight space
        optimizer.step()
        
        #Every so often, let's print out the accuracy
        if step % 1000 == 0:
            
            #Evaluate the network's predictions
            train_output = cnn(torch.unsqueeze(data.train_data, dim=1).type(torch.FloatTensor)[0:200])
            validation_output = cnn(test_x)
            
            train_predictions = torch.max(train_output, 1)[1].data.numpy()
            validation_predictions = torch.max(validation_output, 1)[1].data.numpy()
            
            #Calculate accuracy
            train_accuracy = float((train_predictions == data.train_labels.numpy()[0:200]).astype(int).sum()) / float(train_predictions.size)
            validation_accuracy = float((validation_predictions == test_y.data.numpy()).astype(int).sum()) / float(validation_predictions.size)
            
            print("Epoch: {0} Step: {1}  | Training Accuracy: {2} -- Validation Accuracy: {3}".format(epoch + 1, step, train_accuracy, validation_accuracy))
            
            #save results to list for diagnostic plots
            indices.append(index_counter)
            losses.append(loss.data.numpy())
            train_acc.append(train_accuracy)
            validation_acc.append(validation_accuracy)
            index_counter += 1
            

## Diagnostics

Some common ways to assess the performance of you network.

**Method 1**: Plot out the images with the label predicted for them by the CNN. This should give you an intuition for what features the CNN is basing the classifications on and what features it may be missing. If you notice a pattern in features that are not being picked up on (for example, it misses discriminating features that are similar size), consider adjusting the `kernel_sizes` in your network.

In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(20, 20))

counter = 0
for i in range(5):
    for j in range(5):
        axs[i, j].imshow(test_data.test_data[counter], cmap='gray')
        axs[i, j].set_title('CNN Prediction: {0}'.format(validation_predictions[counter]), fontsize=22)
        counter += 1

fig.tight_layout()
fig.show()

Here we are doing pretty well, but there are a couple images confusing the network. Not surprisingly, these hand-written digits look a little off, so the network should be tuned to get these right. A worthwhile exercise is to augment the above network, restart the notebook kernel, and retrain until you see improvements in accuracy.

**Method 2**: Assessing loss and accuracy as functions of training time. As you may have noticed above, training took a while, but the accuracy seemed to stagnate after a little while. This behavior indicates that we might as well save ourselves some time and stop the training earlier. Here are some plots to visualize this:

In [ ]:
def plot_loss_and_accuracy(indices, losses, train_acc, validation_acc, number_of_training_epochs):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))
    
    fit = np.poly1d(np.polyfit(indices, losses, 50))(indices)
    ax1.plot(indices, losses, color='darkgreen', alpha=0.3)
    ax1.plot(indices, fit, color='darkgreen', lw=2)
    ax1.set_xlabel("Training Step", fontsize=22)
    ax1.set_ylabel("Categorical Cross Entropy Loss", fontsize=22)
    for ii in range(number_of_training_epochs):
        xval = (ii + 1) / number_of_training_epochs * np.max(indices)
        ax1.axvline(x=xval, ls='--', color='black')
        ax1.text(xval - number_of_training_epochs, 0.9, "Epoch {0}".format(ii + 1), horizontalalignment='right', fontsize=18)
    ax1.set_ylim(0.0, 1.0)

    train_fit = np.poly1d(np.polyfit(indices, train_acc, 50))(indices)
    validation_fit = np.poly1d(np.polyfit(indices, validation_acc, 50))(indices)
    ax2.plot(indices, train_acc, alpha=0.3, color='darkorange')
    ax2.plot(indices, validation_acc, alpha=0.3, color='darkblue')
    ax2.plot(indices, train_fit, lw=2, color='darkorange', label="Training Data")
    ax2.plot(indices, validation_fit, lw=2, color='darkblue', label="Validation Data")
    ax2.legend(loc='center right', fontsize=22)
    ax2.set_xlabel("Training Step", fontsize=22)
    ax2.set_ylabel("Accuracy", fontsize=22)
    for ii in range(number_of_training_epochs):
        xval = (ii + 1) / number_of_training_epochs * np.max(indices)
        ax2.axvline(x=xval, ls='--', color='black')
        ax2.text(xval - number_of_training_epochs, 0.82, "Epoch {0}".format(ii + 1), horizontalalignment='right', fontsize=18)
    ax2.set_ylim(0.8, 1.0)

    fig.tight_layout()
    fig.show()
    return      

In [ ]:
plot_loss_and_accuracy(indices, losses, train_acc, validation_acc, number_of_training_epochs)

On the left we plot the loss, which is the quantity the network is attempting to minimize while making its classifications. On the right we evaluate the accuracy on the training and validation sets. The noise level in these curves is normal behavior, as there is a lot of randomness going on under the hood. It is also a good sign that the training and validation accuracies trace each other. If this were not the case, we might start to worry that either our training data was not representative of all the aspects of our validation data, or that the network was overfitting the training data.

For the MNIST dataset, it seems like all of these curves level off after the second epoch, so when tweaking the network, you might consider just terminating the training after two epochs to save yourself some time. The MNIST dataset is very easy to work with though, so in practice you will need many more epochs. As well, we only had 4 total layers in this network; the deeper you make your network, the more epochs your network will require to converge.

**Method 3**: Confusion matrices. The plots below are one of your best tools for assessing points of confusion in your network:

In [ ]:
def convert_to_index(array_categorical):
    array_index = [np.argmax(array_temp) for array_temp in array_categorical]
    print(np.unique(array_index))
    return array_index

def plot_confusion_matrix(cm, ax,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function modified to plots the ConfusionMatrix object.
    Normalization can be applied by setting `normalize=True`.
    
    Code Reference : 
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    
    This script is derived from PyCM repository: https://github.com/sepandhaghighi/pycm
    
    """

    plt_cm = []
    for i in cm.classes :
        row=[]
        for j in cm.classes:
            row.append(cm.table[i][j])
        plt_cm.append(row)
    plt_cm = np.array(plt_cm)
    if normalize:
        plt_cm = plt_cm.astype('float') / plt_cm.sum(axis=1)[:, np.newaxis]     
    ax.imshow(plt_cm, interpolation='nearest', cmap=cmap)
    ax.set_title(title, fontsize=28)
    #ax.colorbar()
    tick_marks = np.arange(len(cm.classes))
    ax.set_xticks(tick_marks)
    ax.set_xticklabels([str(x) for x in cm.classes], fontsize=20)
    ax.set_yticks(tick_marks)
    ax.set_yticklabels([str(x) for x in cm.classes], fontsize=20)
    

    fmt = '.2f' if normalize else 'd'
    thresh = plt_cm.max() / 2.
    for i, j in itertools.product(range(plt_cm.shape[0]), range(plt_cm.shape[1])):
        ax.text(j, i, format(plt_cm[i, j], fmt),
                horizontalalignment="center",
                color="white" if plt_cm[i, j] > thresh else "black",
                fontsize=16)

    
    ax.set_ylabel('Actual', fontsize=24)
    ax.set_xlabel('Predict', fontsize=24)
    return ax

def plot_heatmaps(training_predictions, validation_predictions, training_labels, validation_labels):
    train_true_idx = training_labels.numpy()
    train_pred_idx = training_predictions
    validation_true_idx = validation_labels.numpy()
    validation_pred_idx = validation_predictions
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    
    cm_train = ConfusionMatrix(train_true_idx, train_pred_idx)
    ax1 = plot_confusion_matrix(cm_train, ax1, title="Training Data")
    
    cm_validation = ConfusionMatrix(validation_true_idx, validation_pred_idx)
    ax2 = plot_confusion_matrix(cm_validation, ax2, title="Validation Data")
    
    fig.tight_layout()
    fig.show()
    
    return

The cell below evaluates the predictions on the full training and validaion sets. It may take a minute or two...

In [ ]:
print("Getting all training set predictions")
training_predictions = torch.max(cnn(torch.unsqueeze(data.train_data, dim=1).type(torch.FloatTensor)), 1)[1].data.numpy()
print("Getting all validation set predictions")
validation_predictions = torch.max(cnn(torch.unsqueeze(test_data.test_data, dim=1).type(torch.FloatTensor)), 1)[1].data.numpy()
print("Done!")

In [ ]:
plot_heatmaps(training_predictions, validation_predictions, data.train_labels, test_data.test_labels)

Overall, this network performs very well, evidenced by the prominent main diagonal in these matrices. There is notable confusion between 9 and 4, judging by the largest off-diagonal elements. We might consider starting the convolutional layers with a larger kernel, since the top of the 9 (which is the only real difference between a 9 and a 4) is typically larger than 5 pixels wide in these images. 

## Weaknesses of CNNs

Convolution is not a rotation-, scale-, or translation-invariant operation.

The reason for this fun-to-deal-with property is the convolution kernels operate on local features. See the example below, which is illustrating translation confusing a CNN.

<img src="./images/invariance.png" alt="ljag" width="600"/>

[Image Source](https://github.com/rasbt/stat479-deep-learning-ss19/blob/master/L13_intro-cnn/L13_intro-cnn-part1_slides.pdf)

In the convolved representation of this image of the number 5, different regions are being activated, which leads to different information being passed through the network even though this is the same real object. You can imagine that the network would be confused in similar fashion by different sized numbers and rotated numbers.

As a fun test of this weakness, let's try to confuse our CNN beyond belief.

### Can our CNN handle rotation?

In [ ]:
#Rotation using scipy.ndimage.rotate will expand the image by filling in zeros, so we need to trim it back down
def zoom_after_rotation(x, desired_shape=(28, 28)):
    current_width = np.shape(x)[0]
    assert current_width > desired_shape[0]
    num_to_trim = int(round((np.shape(x)[0] - desired_shape[0]) / 2))
    return x[num_to_trim: - num_to_trim, num_to_trim: - num_to_trim]

In [ ]:
#Rotate images in test set
rotated_test_images_30_deg_expanded = [ndimage.rotate(x, angle=30.0) for x in test_data.test_data[0:5]]
rotated_test_images_30_deg = np.array([zoom_after_rotation(x)/255. for x in rotated_test_images_30_deg_expanded])

rotated_test_images_90_deg = np.array([ndimage.rotate(x, angle=90.0) for x in test_data.test_data[0:5]])

rotated_test_images_180_deg = np.array([ndimage.rotate(x, angle=180.0) for x in test_data.test_data[0:5]])

In [ ]:
#Get CNN predictions on rotated images
rotated_test_images_30_deg_predictions = torch.max(cnn(torch.unsqueeze(torch.from_numpy(rotated_test_images_30_deg), dim=1).type(torch.FloatTensor)), 1)[1].data.numpy()
rotated_test_images_90_deg_predictions = torch.max(cnn(torch.unsqueeze(torch.from_numpy(rotated_test_images_90_deg), dim=1).type(torch.FloatTensor)), 1)[1].data.numpy()
rotated_test_images_180_deg_predictions = torch.max(cnn(torch.unsqueeze(torch.from_numpy(rotated_test_images_180_deg), dim=1).type(torch.FloatTensor)), 1)[1].data.numpy()

In [ ]:
#Plot results
images = [rotated_test_images_30_deg, rotated_test_images_90_deg, rotated_test_images_180_deg]
preds = [rotated_test_images_30_deg_predictions, rotated_test_images_90_deg_predictions, rotated_test_images_180_deg_predictions]
titles = ["30 Degree Rotation", "90 Degree Rotation", "180 Degree Rotation"]
for ii in range(3):
    fig, axs = plt.subplots(1, 5, figsize=(20, 4))
    for jj in range(5):
        axs[jj].imshow(images[ii][jj], cmap='gray')
        axs[jj].set_xlabel("CNN Prediction: {0}".format(preds[ii][jj]), fontsize=20)
    fig.suptitle(titles[ii], fontsize=24)
    fig.show()

So clearly, a CNN cannot handle rotated images if it was not exposed to them during training.

### What about changes to the scale of the image?

In [ ]:
# Zoom out on images in test set
zoom_out_images = np.array([misc.imresize(np.pad(x, 14, mode='minimum'), (28,28)) for x in test_data.test_data[0:5]])

# Zoom in on images in test set
zoom_in_images = np.array([misc.imresize(x[6:-5, 6:-5], (28,28)) for x in test_data.test_data[0:5]])

In [ ]:
# Get predictions from CNN
zoom_out_pred = torch.max(cnn(torch.unsqueeze(torch.from_numpy(zoom_out_images), dim=1).type(torch.FloatTensor)), 1)[1].data.numpy()
zoom_in_pred = torch.max(cnn(torch.unsqueeze(torch.from_numpy(zoom_in_images), dim=1).type(torch.FloatTensor)), 1)[1].data.numpy()

In [ ]:
# Plot results
images = [zoom_out_images, zoom_in_images]
preds = [zoom_out_pred, zoom_in_pred]
titles = ["Zoomed Out", "Zoomed In"]
for ii in range(2):
    fig, axs = plt.subplots(1, 5, figsize=(20, 4))
    for jj in range(5):
        axs[jj].imshow(images[ii][jj], cmap='gray')
        axs[jj].set_xlabel("CNN Prediction: {0}".format(preds[ii][jj]), fontsize=20)
    fig.suptitle(titles[ii], fontsize=24)
    fig.show()

So clearly the CNN is sensitive to scale of the objects in the images, as the size of the features it can pick up is dependent on both the features sizes in the training set and the user-specified kernel dimensions. 

For rotations, scale changes, and translations, the problem can be approached from a preprossing standpoint. The preprocessing approach is to augment the training set to include rotated images, translated images, zoomed in or out images, or even partial images. This step will allow the network to recognize the same object in multiple representations and is an essential first step of most real world applications of CNNs.

# Summary

In this notebook, we

- Saw how to work with PyTorch
- Built a CNN
- Trained our CNN using a CPU
- Assessed the performance of our CNN
- Found weaknesses in the CNN approach

### Topics for later tutorials

- Save a trained model
- Utilize a GPU
- Batch normalization
- Autoencoders
- Multicolor image classification
- Regression with CNNs
- Advanced preprocessing
    - cropping
    - centering
    - contour detection
    - image augmentation